This notebook explores AI agents, specifically using the LangChain framework. Think of an agent as a smart system that uses a Large Language Model (LLM) not just to answer questions, but to reason and decide on a sequence of actions. To do this, we give the agent access to tools—like a search engine, a calculator, or even custom Python code. The agent's job is to figure out which tools to use, and in what order, to accomplish a task it couldn't solve on its own.

### Getting Started

In [ ]:
!pip install -q langchain-google-genai google-genai langchain

In [ ]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
import requests
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

# Get the API key from Colab secrets
API_KEY = userdata.get('GOOGLE_API_KEY')


### Test your API KEY

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=API_KEY)
try:
    response = llm.invoke("What is LLM?")
    print(response)
except Exception as e:
    print(f"An error occurred: {e}")


### Python function as a tool

In [ ]:
# Currency converter function
def convert_currency(input_text: str) -> str:
    """
    Input format: '100 USD to INR'
    """
    try:
        parts = input_text.strip().upper().split()
        amount = float(parts[0])
        from_currency = parts[1]
        to_currency = parts[3]

        # Call exchange rate API
        url = f"https://api.exchangerate-api.com/v4/latest/{from_currency}"
        response = requests.get(url)
        data = response.json()

        rate = data["rates"].get(to_currency)
        if rate is None:
            return f"Conversion rate not found for {to_currency}."

        converted = amount * rate
        return f"{amount} {from_currency} = {converted:.2f} {to_currency}"

    except Exception as e:
        return f"Error: {str(e)}. Format should be like '100 USD to INR'"

### Define your fist Agent

In [ ]:
from langchain.agents import create_agent

In [ ]:
agent = create_agent(
    model=llm,
    tools=[convert_currency],
    system_prompt="You are a currency conversion finance expert",
)

### Test your Agent

In [ ]:
def print_readable_agent_output(agent_output):
    """
    Parses LangChain agent output and prints it cleanly.
    """
    messages = agent_output.get("messages", [])

    print("--- Agent Execution Trace ---\n")

    for msg in messages:
        # 1. Handle User Input
        if isinstance(msg, HumanMessage):
            print(f"👤 USER: \n   {msg.content}\n")

        # 2. Handle AI Reasoning & Tool Calls
        elif isinstance(msg, AIMessage):
            # If the AI decided to call a tool (Check for tool_calls)
            if msg.tool_calls:
                tool_name = msg.tool_calls[0]['name']
                tool_args = msg.tool_calls[0]['args']
                print(f"🤖 AI (Thought): \n   Decided to use tool '{tool_name}' with args: {tool_args}\n")

            # If the AI provided the final answer (No tool calls)
            else:
                print(f"✅ FINAL ANSWER: \n   {msg.content}\n")

        # 3. Handle Tool Outputs
        elif isinstance(msg, ToolMessage):
            print(f"⚙️ TOOL OUTPUT: \n   {msg.content}\n")

    print("-----------------------------")

In [ ]:
query ="Convert 250 EUR to JPY"
print_readable_agent_output(agent.invoke({"messages": [{"role": "user", "content": query}]}))

### Adding another tool

In [ ]:
# One more tool
def multiply(text: str) -> str:
    '''
    Multiplies two numbers provided as a space-separated string."
    '''
    a, b = map(int, text.split())
    return str(a * b)

### Redefine Agent

In [ ]:
agent = create_agent(
    model=llm,
    tools=[convert_currency, multiply],
    system_prompt="You are a currency conversion finance expert and do math operations ",
)

### Test the Agent with Multiple Tools

In [ ]:
query ="what is 5 times 4? "
print_readable_agent_output(agent.invoke({"messages": [{"role": "user", "content": query}]}))

In [ ]:
query ="100 indian rupees equals to how many US ?"
print_readable_agent_output(agent.invoke({"messages": [{"role": "user", "content": query}]}))